In [1]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Churn Prediction with SVM") \
    .getOrCreate()

In [2]:
# Đọc dữ liệu từ file CSV
customer_data = spark.read.csv('C:/Users/PC/Desktop/Do_an_Big_data/PythonCodes/work/data/Churn_Modelling_FE.csv', header=True, inferSchema=True)

# Loại bỏ các cột không cần thiết
dataset = customer_data.drop('RowNumber', 'CustomerId', 'Surname')

In [3]:
from pyspark.ml.feature import VectorAssembler

# Tách features và label
feature_columns = [col for col in dataset.columns if col != 'Exited']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(dataset)

# Chọn cột features và label
data = data.select("features", "Exited")

In [4]:
from pyspark.ml.feature import StandardScaler

# Chuẩn hóa dữ liệu
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(data)
scaled_data = scaler_model.transform(data)

# Chọn cột đã chuẩn hóa và label
scaled_data = scaled_data.select("scaled_features", "Exited").withColumnRenamed("scaled_features", "features")

In [5]:
# Chia dữ liệu thành tập train và test
train_data, test_data = scaled_data.randomSplit([0.8, 0.2], seed=0)

# In kích thước tập dữ liệu
print(f"Train data shape: {train_data.count()}, Test data shape: {test_data.count()}")

Train data shape: 7602, Test data shape: 1966


In [7]:
from pyspark.sql.functions import col

# Đếm số lượng mẫu của từng lớp trong tập train
train_counts = train_data.groupBy("Exited").count().collect()
majority_class_count = max([row['count'] for row in train_counts])
minority_class_count = min([row['count'] for row in train_counts])

# Lấy dữ liệu lớp thiểu số và lớp đa số
minority_class = train_data.filter(col("Exited") == 1)
majority_class = train_data.filter(col("Exited") == 0)

# Oversampling lớp thiểu số
oversampling_ratio = float(majority_class_count / minority_class_count)  # Chuyển sang float
balanced_train_data = majority_class.union(minority_class.sample(withReplacement=True, fraction=oversampling_ratio, seed=0))

# Kiểm tra số lượng sau khi cân bằng
balanced_train_data.groupBy("Exited").count().show()

+------+-----+
|Exited|count|
+------+-----+
|     0| 6134|
|     1| 6108|
+------+-----+



In [10]:
# Huấn luyện mô hình SVM
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col
svm = LinearSVC(featuresCol="features", labelCol="Exited", maxIter=100, regParam=0.1)
svm_model = svm.fit(balanced_train_data)

# Dự đoán
predictions = svm_model.transform(test_data)

# Đánh giá
evaluator = MulticlassClassificationEvaluator(labelCol="Exited", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Lưu mô hình
svm_model.save("C:/Users/PC/Desktop/Do_an_Big_data/PythonCodes/work/models/svm/spark_svm_model")

# Tắt SparkSession
spark.stop()

Accuracy: 0.7314
Precision: 0.7998
Recall: 0.7314
F1-score: 0.7515
